## QR Extractor Demo

This is a simple demonstration script to show *Observation-based extraction*  using Questionnaire and QuestionnaireResponse Data.

### Background

**Observation-based extraction**

This is the simplest of the extraction mechanisms. It leverages the same data elements as are used for the Observation-based population mechanism. It takes advantage of the fact that most questions in the healthcare space typically correspond to the value element of an Observation. It also takes advantage of the Questionnaire.item.code element that identifies what a concept each question or group corresponds to.

To use this method:

- Include the item.code element on each question to be extracted. Typically, this will be a LOINC code, but in some jurisdictions/environments, SNOMED CT or other codes may be relevant
- Groups can also have an item.code present - this might represent the code of the a panel or the Observation.code of an Observation with no value but with multiple Observation.component elements. Child question items can then assert the item.code of the "member-of" Observations or the Observation.component.code values
- To signal that the item.code is intended for use in extraction (as opposed to just providing metadata about the Questionnaire item, the `questionnaire-observationLinkPeriod` extension must also be included. This extension indicates the period of time over which to search for matching observations.
- Multiple item.code elements might be present. If so, each are considered one of the Observation.code Codings in the resulting extracted Observation.
                                                                  
### Mapping to Observation         
                                                                  
- Observation.basedOn and Observation.partOf - copy from QuestionnaireResponse elements of the same name
- Observation.status - set to 'final'
- Observation.category - if this can be inferred from any of the Questionnaire.item.code values or from known context of the Questionnaire itself, then fill it in, otherwise omit.
- Observation.code - add all the Questionnaire.item.code values as Observation.code.coding instances
- Observation.subject - set to QuestionnaireResponse.subject
- Observation.encounter - set to QuestionnaireResponse.context (if an Encounter)
- Observation.effectiveDateTime - set to QuestionnaireResponse.authored.

Note, this is an inference. It is important that the question text implies that the value is 'current' not 'historical' for this to be safe - otherwise don't include the 'observationLinkPeriod' extension that marks the question as appropriate for population and extraction.

- Observation.issued - set to QuestionnaireResponse.authored
- Observation.performer - set to QuestionnaireResponse.author
- Observation.value[x] - set to QuestionnaireResponse.item.answer.value[x]
- Observation.derivedFrom - set to a reference to the QuestionnaireResponse
- Observation.interpretation and Observation.referenceRange - if these can be inferred from the QuestionnaireResponse.item.code (and for interpretation the answer value too), they can be populated, otherwise omit

### imports and constants

In [2]:
from fhirclient.r4models import questionnaire as Q
from fhirclient.r4models import questionnaireresponse as QR
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
from json import dumps, loads, load, dump
from requests import get, post, put
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from ipywidgets import Layout
from pprint import pprint

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

params = dict()

R4fhir_server = 'http://hapi.fhir.org/baseR4'

In [3]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='80%',
                   height = '550px')


w = widgets.Textarea(
    placeholder='copy and paste QR example here',
    description='QR Example',
    disabled=False,
    layout=box_layout
)


display(Markdown('### Copy and Paste QR Example from which Responses are to be extracted  (examples test files can be found [here](#))'),w)
               
display(Markdown('### Scroll down to see the Extractions steps and FHIR Observation'))

### Copy and Paste QR Example from which Responses are to be extracted  (examples test files can be found [here](#))

Textarea(value='', description='QR Example', layout=Layout(align_items='stretch', border='solid', display='fle…

### Scroll down to see the Extractions steps and FHIR Observation

### Inspect QR for Q url and Fetch test Q from FHIR Server

- instantiate copy and pasted QR as a QR pyfhirclient model
- fetch Q and instantiate as a Q pyfhirclient model

In [4]:
#w.close()
qr = QR.QuestionnaireResponse(loads(w.value))
#print(dumps(qr.as_json(), indent = 4))

display(Markdown(f'### Fetching Questionnaire `[base]Questionniare?questionnaire={{QR.questionnaire}}`'))
r_id = qr.questionnaire.split('/')[-1]
display(Markdown(f'**QR.questionnaire** = {qr.questionnaire}'))                        
r = get(f'{R4fhir_server}/Questionnaire/{r_id}', params = params, headers = headers)   # return r.status_code
display(Markdown(f'**Status** = {r.status_code}'))
q = Q.Questionnaire(r.json())
print(dumps(q.as_json(), indent = 4))                             

### Fetching Questionnaire `[base]Questionniare?questionnaire={QR.questionnaire}`

**QR.questionnaire** = http://devdays2019/Questionnaire/devdays-q-1

**Status** = 200

{
    "id": "devdays-q-1",
    "meta": {
        "lastUpdated": "2019-06-09T07:31:29.583+00:00",
        "profile": [
            "http://hl7.org/fhir/uv/sdc/StructureDefinition/sdc-questionnaire|3.5.0"
        ],
        "versionId": "3"
    },
    "item": [
        {
            "code": [
                {
                    "code": "[H1]",
                    "display": "Answer Me These Questions Three",
                    "system": "http://devdays2019/q"
                }
            ],
            "item": [
                {
                    "extension": [
                        {
                            "url": "http://hl7.org/fhir/StructureDefinition/questionnaire-minOccurs",
                            "valueInteger": 1
                        }
                    ],
                    "code": [
                        {
                            "code": "[Q1]",
                            "display": "1   WHAT... is your name ?",
                            "system

### Inspect Q and choose which items get exported

In [5]:
q_items=[]
for i in q.item:  # list all the main items
    for j in i.item:    # list all the 2nd level items         
        q_items.append((f'linkId = {j.linkId}, text = {j.text}',j.code[0].code))
#print(q_items)

exports = widgets.SelectMultiple(
    options=q_items,
    description='Choose items to Extract', 
    style={'description_width': 'initial'},
    layout={'width': 'initial'},
    disabled=False
)
display(exports)

SelectMultiple(description='Choose items to Extract', layout=Layout(width='initial'), options=(('linkId = /[H1…

### Update Q with the link period extension and gather data for extraction

In [6]:
#exports.close()
extract_items = dict()

x_link_period = X.Extension(            {
              "url": "http://hl7.org/fhir/StructureDefinition/questionnaire-observationLinkPeriod",
              "valueDuration": {
                "value": 0,
                "system": "http://unitsofmeasure.org",
                "code": "s"
              }
            })


for i in q.item:
     for j in i.item:
        if j.code[0].code in exports.value:
            #add link period extension
            if not j.extension or 'http://hl7.org/fhir/StructureDefinition/questionnaire-observationLinkPeriod' not in [x.url for x in j.extension]:
                try:
                    j.extension.append(x_link_period)
                except AttributeError:
                    j.extension = [x_link_period]
            extract_items[j.linkId]=(j.code[0],j.text) #extract_items for conversion

display(Markdown('''### Q data to be exported into Observations
                 (Python dict data structure)'''))
pprint(extract_items)

display(Markdown('### Updated Q with observationLinkPeriod added to items to be exported'))
            
print(dumps(q.as_json(), indent=4))

### Q data to be exported into Observations
                 (Python dict data structure)

{'/[H1]/[Q2]': (<fhirclient.r4models.coding.Coding object at 0x01313510>,
                'WHAT... is your Quest')}


### Updated Q with observationLinkPeriod added to items to be exported

{
    "id": "devdays-q-1",
    "meta": {
        "lastUpdated": "2019-06-09T07:31:29.583+00:00",
        "profile": [
            "http://hl7.org/fhir/uv/sdc/StructureDefinition/sdc-questionnaire|3.5.0"
        ],
        "versionId": "3"
    },
    "item": [
        {
            "code": [
                {
                    "code": "[H1]",
                    "display": "Answer Me These Questions Three",
                    "system": "http://devdays2019/q"
                }
            ],
            "item": [
                {
                    "extension": [
                        {
                            "url": "http://hl7.org/fhir/StructureDefinition/questionnaire-minOccurs",
                            "valueInteger": 1
                        }
                    ],
                    "code": [
                        {
                            "code": "[Q1]",
                            "display": "1   WHAT... is your name ?",
                            "system

### Export these items from QR into the Observation template

- Observation template for this simple demo is a Python dict

In [23]:
derived_obs=[]
for link_id, (item_code,item_text) in extract_items.items():
    #print( item_code, item_text)
    value_pair = [j.answer[0] for j in qr.item[0].item if j.linkId == link_id][0]
    #print(value_pair)
    (k, v), = value_pair.as_json().items()
    v = {'coding': [v]} if k == 'valueCoding' else v
    k = 'valueCodeableConcept' if k == 'valueCoding' else k
    display(Markdown(f'### Question = {item_text}, {k} = {v}'))
    o_id = f'devdays-o{link_id.lower().replace("/","-").replace("[","").replace("]","")}'
    
    '''{{
      "resourceType": "Observation",
      "id": "{o_id}",
      "status": "final",
      "code": {{
      "coding":[
            {dumps(item_code.as_json())}
        ],
        "text": "{item_text}"
      }},
      "subject": {dumps(qr.subject.as_json())},
      "effectiveDateTime": {dumps(qr.authored.as_json())},
      "issued": {dumps(qr.authored.as_json())},
      "{k}": {v},
      "derivedFrom": [{{"reference":"QuestionnaireResponse/{qr.id}"}}]
    }}'''
    
    o_dict = {
          'resourceType': 'Observation',
          'id': o_id,
          'status': 'final',
          'code': {
          'coding':[
                item_code.as_json()
            ],
            'text': item_text
          },
          'subject': qr.subject.as_json(),
          'effectiveDateTime': qr.authored.as_json(),
          'issued': qr.authored.as_json(),
           k: v,
          'derivedFrom': [{'reference':f'QuestionnaireResponse/{qr.id}'}]
        }

    #print(o_dict)
    o_json = dumps(o_dict, indent=4)
    derived_obs.append(tuple([o_id,o_json]))
    print(o_json)
    

### Question = WHAT... is your Quest, valueCodeableConcept = {'coding': [{'code': 'c3', 'display': 'Fetch a shrubbery'}]}

{
    "resourceType": "Observation",
    "id": "devdays-o-h1-q2",
    "status": "final",
    "code": {
        "coding": [
            {
                "code": "[Q2]",
                "display": "WHAT... is your Quest",
                "system": "http://devdays2019/q"
            }
        ],
        "text": "WHAT... is your Quest"
    },
    "subject": {
        "display": "Markus Gutmann",
        "reference": "Patient/28ee432c-fd33-4101-a767-5ebd7087a187"
    },
    "effectiveDateTime": "2019-06-08T16:11:49-07:00",
    "issued": "2019-06-08T16:11:49-07:00",
    "valueCodeableConcept": {
        "coding": [
            {
                "code": "c3",
                "display": "Fetch a shrubbery"
            }
        ]
    },
    "derivedFrom": [
        {
            "reference": "QuestionnaireResponse/devdays-qr-1"
        }
    ]
}


### Validate Observations

In [24]:
for o_id,o_json in derived_obs:
    r =  post(f'{R4fhir_server}/Observation/$validate', params = params, headers = headers, data = o_json.encode('utf-8'))   # return r.status_code
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

WARNING,[Observation],All observations should have a performer
WARNING,[Observation],A resource should have narrative for robust management [text.div.exists()]
WARNING,[Observation],A resource should have narrative for robust management [text.div.exists()]


### Save Observations

In [25]:
for o_id,o_json in derived_obs:
    r =  put(f'{R4fhir_server}/Observation/{o_id}', params = params, headers = headers, data = o_json.encode('utf-8'))   # return r.status_code
    display(HTML(f'<h1>Create/Update: {R4fhir_server}/Observation/{o_id}</h1><h3>Status Code = {r.status_code}</h3> ,<pre>{dumps(r.json(),indent=4)}</pre>'))